In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import os
import json

## scrape

### 1.1 mapping

In [2]:
links = []
for page in range(1,201,1):
    url = f'https://www.boligsiden.dk/tilsalg?sortAscending=true&page={page}'
    links.append(url)

In [3]:
links

['https://www.boligsiden.dk/tilsalg?sortAscending=true&page=1',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=2',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=3',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=4',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=5',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=6',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=7',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=8',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=9',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=10',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=11',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=12',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=13',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=14',
 'https://www.boligsiden.dk/tilsalg?sortAscending=true&page=15',
 'https://www.boligsiden.dk/tilsal

In [4]:
# Create a new list
houses = []
for url in links:
    # Connects to site
    response = requests.get(url, headers={'name':'He SHI','email':'pvs237@alumni.ku.dk'})
    
    #Parse data with BeautifulSoup 
    soup = BeautifulSoup(response.content, 'lxml')

    # Identify houses to scrape by inspecting site 
    houses += soup.find_all('div', class_ = 'overflow-hidden relative shadow-card rounded-sm grid grid-cols-1 grid-rows-1 sm:grid-cols-12 sm:grid-rows-1 border border-gray-100 bg-background')

    # sleep for 0.5 seconds
    time.sleep(0.5) #Sleep for 0.5 seconds

In [5]:
houses
len(houses)

9999

We then create a list of URLs that we want to scrape

In [6]:
# Create an empty list
list_of_house_urls = []

# Creating a loop that appends the article url to the list above
for i in range(len(houses)):
    list_of_house_urls.append(houses[i].find('a')['href'])

In [7]:
list_of_house_urls

['/adresse/strandparken-46-4591-foellenslev-03261666__46_______?udbud=486768e7-0997-4d9e-a7ee-7b2b1cf3ae7a',
 '/adresse/kalkbraenderihavnsgade-4a-1-tv-2100-koebenhavn-oe-01013516__4a__1__tv?udbud=4b014ec2-a956-4372-826d-7aaed8efd7dc',
 '/adresse/niels-andersens-vej-56-2900-hellerup-01570587__56_______?udbud=ef76ca7a-15d3-4969-89a6-7df442e5d2cb',
 '/adresse/buskelundengen-72-8600-silkeborg-07400202__72_______?udbud=2c230f89-d2e2-4eaa-88f5-2fd7b88a741d',
 '/adresse/helga-pedersens-gade-1-2-3-8000-aarhus-c-07513178___1__2___3?udbud=b787bbe7-fe0e-45b4-85c6-c5d546c57dfb',
 '/adresse/noeddevaenget-20-7100-vejle-06301757__20_______?udbud=8485421a-fb44-4c7d-be84-dd60e0af6006',
 '/adresse/englandsvej-38b-5-2300-koebenhavn-s-01011600_38b__5____?udbud=186552d9-061a-43c5-8cad-ca174827c27f',
 '/viderestillingaabenthus/e2037fc7-c445-4514-8d60-c9fc757f81ee',
 '/viderestillingaabenthus/03ab0f46-b577-4c99-b615-648628a3009e',
 '/adresse/vognmandsmarken-12-0-mf-4000-roskilde-02659358__12_st__mf?udbud=918

Some of the links are not to articls, then we...

In [8]:
list_of_house_urls_final = []
for link in list_of_house_urls:
    if '/adresse' in link: #All article URLs have this string in them, so we restrict on it being in the URL
        list_of_house_urls_final.append(link)

len(list_of_house_urls_final)

7619

### 1.2 Downloading + 1.3 Parsing

In [9]:
# Define the log function to gather the log information
def log(response,logfile,output_path=os.getcwd()):
    # Open or create the csv file
    if os.path.isfile(logfile): #If the log file exists, open it and allow for changes     
        log = open(logfile,'a')
    else: #If the log file does not exist, create it and make headers for the log variables
        log = open(logfile,'w')
        header = ['timestamp','status_code','length','output_file']
        log.write(';'.join(header) + "\n") #Make the headers and jump to new line
        
    # Gather log information
    status_code = response.status_code #Status code from the request result
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time())) #Local time
    length = len(response.text) #Length of the HTML-string
    
    # Open the log file and append the gathered log information
    with open(logfile,'a') as log:
        log.write(f'{timestamp};{status_code};{length};{output_path}' + "\n") #Append the information and jump to new line

In [10]:
# Creatig empty list for the infomation we want to extract for every article
house_data_list=[]
logfile = 'log.csv'
for i in range(7619): #len(list_of_house_urls_final)
    try:
        # This time we scrape for each house in the url list we created before
        url = 'https://www.boligsiden.dk' + list_of_house_urls_final[i] #The scraped links are relative, so we need to add the base url
        response = requests.get(url)
        soup = BeautifulSoup(response.content,'lxml')

        # Logging
        log(response,logfile)

        # creat a dictionary
        house_data = {}
    
        # Append price to list
        temp = soup.find('div', class_='flex flex-row justify-center items-center space-x-2')
        temp = temp.text.strip()
        house_data['price'] = temp
    
        # Append address to list
        temp = soup.find('div', class_='font-bold text-sm md:text-base') 
        temp = temp.text.strip()
        house_data['address'] = temp
    
        # Append area to list
        temp = soup.find('div', class_='mt-1 text-xs md:text-sm text-gray-600')
        temp = temp.text.strip()
        house_data['area'] = temp

        # Append city to list
        temp = soup.find('span', class_='inline-flex')
        temp = temp.text.strip()
        house_data['city'] = temp

        # Append type to list
        temp = soup.find('span', class_='text-black text-sm pr-2')
        temp = temp.text.strip()
        house_data['type'] = temp

        # Append other data to list 
        temp = soup.find('div', class_='py-5 px-2 md:px-6 grid text-sm grid-cols-2')
        temp = temp.text.strip()
        house_data['data'] = temp

        # Find days till sale
        temp = soup.find('div', class_='pb-0.5 border-dashed border-b border-gray-300 cursor-pointer text-sm text-gray-800') 
        temp = temp.text.strip()
        house_data['saledays'] = temp

        # Find energy ranking
        temp = soup.find('svg', id='Lag_1') 
        temp = temp.text.strip()
        house_data['energy'] = temp

        # Append the house data dictionary to the list
        house_data_list.append(house_data)
    
        # Time sleep
        time.sleep(0.5) #Sleep for 0.5 seconds

     

    except Exception as e:
        print(url) #Print url
        print(e) #Print error
        continue


https://www.boligsiden.dk/adresse/buskelundengen-72-8600-silkeborg-07400202__72_______?udbud=2c230f89-d2e2-4eaa-88f5-2fd7b88a741d
'NoneType' object has no attribute 'text'
https://www.boligsiden.dk/adresse/lilleaavej-73-7500-holstebro-06613700__73_______?udbud=1b89f25a-856e-45da-bcfa-03e1a243f6f1
'NoneType' object has no attribute 'text'
https://www.boligsiden.dk/adresse/helga-pedersens-gade-109-8-th-8000-aarhus-c-07513178_109__8__th?udbud=54a51d3c-33c1-465a-b92b-f58fdc670f6f
'NoneType' object has no attribute 'text'
https://www.boligsiden.dk/adresse/jeanettevej-6-4593-eskebjerg-03260769___6_______?udbud=08872096-327f-4806-b642-b181106d6f89
'NoneType' object has no attribute 'text'
https://www.boligsiden.dk/adresse/jaettevaenget-5-3390-hundested-02600673___5_______?udbud=5f038e29-b1a8-4033-a0d5-83c622f47c49
'NoneType' object has no attribute 'text'
https://www.boligsiden.dk/adresse/barsidalvej-21-7600-struer-06710565__21_______?udbud=1ffb26cf-a2e1-4dc6-ba3e-06d96fca7957
'NoneType' obje

In [35]:
house_data_list

[{'price': '1.275.000 kr.',
  'address': 'Strandparken 46',
  'area': 'Havnsø, 4591 Føllenslev',
  'city': 'Vest- og Sydsjælland|',
  'type': 'Villa/Fritidsbolig',
  'data': '170 m²Grund: 1019 m²2 værelserEjerudg.: 2.598 kr/mdOpført 1978Se flere detaljer',
  'saledays': 'Til salg i alt: 202 dage',
  'energy': 'Energimærke D'},
 {'price': '5.095.000 kr.',
  'address': 'Kalkbrænderihavnsgade 4A, 1. tv.',
  'area': '2100 København Ø',
  'city': 'Byen København|',
  'type': 'Ejerlejlighed',
  'data': '94 m²Grund: - m²3 værelserEjerudg.: 2.389 kr/mdOpført 2017Se flere detaljer',
  'saledays': 'Til salg i alt: 0 dag',
  'energy': 'Energimærke A2015'},
 {'price': '14.750.000 kr.',
  'address': 'Niels Andersens Vej 56',
  'area': '2900 Hellerup',
  'city': 'Københavns omegn|',
  'type': 'Villa',
  'data': '248 m²Grund: 984 m²8 værelserEjerudg.: 8.643 kr/mdOpført 1941Se flere detaljer',
  'saledays': 'Til salg i alt: 255 dage',
  'energy': 'Energimærke E'},
 {'price': '4.498.000 kr.',
  'addres

In [36]:
df = pd.DataFrame(house_data_list)

In [37]:
csv_file_path = 'original_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-16')

### 1.4 Cleaning

In [51]:
csv_file_path = 'original_data.csv'

df = pd.read_csv(csv_file_path, encoding='utf-16')

print(df.head())

            price                           address                     area  \
0   1.275.000 kr.                   Strandparken 46  Havnsø, 4591 Føllenslev   
1   5.095.000 kr.  Kalkbrænderihavnsgade 4A, 1. tv.         2100 København Ø   
2  14.750.000 kr.            Niels Andersens Vej 56            2900 Hellerup   
3   4.498.000 kr.     Helga Pedersens Gade 1, 2. 3.            8000 Aarhus C   
4   2.850.000 kr.                    Nøddevænget 20               7100 Vejle   

                    city                type  \
0  Vest- og Sydsjælland|  Villa/Fritidsbolig   
1        Byen København|       Ejerlejlighed   
2      Københavns omegn|               Villa   
3            Østjylland|       Ejerlejlighed   
4            Sydjylland|               Villa   

                                                data  \
0  170 m²Grund: 1019 m²2 værelserEjerudg.: 2.598 ...   
1  94 m²Grund: - m²3 værelserEjerudg.: 2.389 kr/m...   
2  248 m²Grund: 984 m²8 værelserEjerudg.: 8.643 k...   
3  107

In [52]:
import re



def extract_data_info(entry):
    living_space = re.search(r'(\d+)\s*m²Grund:', entry)
    ground_space = re.search(r'Grund:\s*(\d+)\s*m²', entry)
    rooms = re.search(r'm²\s*(\d+)\s*værelserEjerudg.:', entry)
    owner_expenses = re.search(r'værelserEjerudg.:\s*([\d\.]+)\s*kr/mdOpført', entry)
    year = re.search(r'kr/mdOpført\s*(\d+)', entry)
    
    living_space = int(living_space.group(1)) if living_space else None
    ground_space = int(ground_space.group(1)) if ground_space else None
    rooms = int(rooms.group(1)) if rooms else None
    owner_expenses = float(owner_expenses.group(1)) if owner_expenses else None
    year = int(year.group(1)) if year else None
    
    return living_space, ground_space, rooms, owner_expenses, year

# create df2 to save new data
df2 = pd.DataFrame(columns=['living_space', 'ground_space', 'rooms', 'owner_expenses', 'year'])


for entry in data_list:
    living_space, ground_space, rooms, owner_expenses, year = extract_data_info(entry)
    df2 = df2.append({'living_space': living_space, 'ground_space': ground_space, 'rooms': rooms, 'owner_expenses': owner_expenses, 'year': year},
                   ignore_index=True)



C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3060736365.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'living_space': living_space, 'ground_space': ground_space, 'rooms': rooms, 'owner_expenses': owner_expenses, 'year': year},
C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3060736365.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'living_space': living_space, 'ground_space': ground_space, 'rooms': rooms, 'owner_expenses': owner_expenses, 'year': year},
C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3060736365.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({'living_space': living_space, 'ground_space': ground_space, 'rooms': rooms, 'owner_e

In [53]:
#merge
df = pd.concat([df, df2], axis=1)
df.drop('data', axis=1, inplace=True)
print(df)

                price                           address  \
0       1.275.000 kr.                   Strandparken 46   
1       5.095.000 kr.  Kalkbrænderihavnsgade 4A, 1. tv.   
2      14.750.000 kr.            Niels Andersens Vej 56   
3       4.498.000 kr.     Helga Pedersens Gade 1, 2. 3.   
4       2.850.000 kr.                    Nøddevænget 20   
...               ...                               ...   
3750  1.750.000 kr.5%        Prins Haralds Allé 101, 1.   
3751      990.000 kr.                       Nissumvej 4   
3752    1.695.000 kr.                    Baunetoften 12   
3753      650.000 kr.                        Aalevej 40   
3754    8.500.000 kr.                 Vestre Skovvej 30   

                          area                   city                type  \
0      Havnsø, 4591 Føllenslev  Vest- og Sydsjælland|  Villa/Fritidsbolig   
1             2100 København Ø        Byen København|       Ejerlejlighed   
2                2900 Hellerup      Københavns omegn|       

In [54]:
df[['area_code', 'area_name']] = df['area'].str.split(' ', 1, expand=True)
df.drop(columns=['area'], inplace=True)
df['city'] = df['city'].str.replace('|', '')
df['price'] = df['price'].str.replace('.', '')
df['price'] = df['price'].str.replace(r'\s*kr.*', '', regex=True)
df['saledays'] = df['saledays'].str.extract(r'(\d+)').astype(float)

C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3029451689.py:1: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df[['area_code', 'area_name']] = df['area'].str.split(' ', 1, expand=True)
C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3029451689.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['city'] = df['city'].str.replace('|', '')
C:\Users\Archie\AppData\Local\Temp\ipykernel_13068\3029451689.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price'] = df['price'].str.replace('.', '')


In [55]:
df['ground_space'] = df['ground_space'].fillna(0)
df = df.dropna()   

In [56]:
type_counts = df['type'].value_counts()
print(type_counts)

Villa                           2157
Ejerlejlighed                    701
Rækkehus                         343
Landejendom                      173
Villa/Fritidsbolig               155
Villalejlighed                    30
Landejendom/Villa                 20
Villa/Landejendom                 19
Fritidsbolig                      11
Landejendom/Landejendom            8
Ejerlejlighed/Fritidsbolig         8
Rækkehus/Fritidsbolig              8
Rækkehus/Villa                     5
Villa/Helårsgrund                  4
Ejerlejlighed/Rækkehus             3
Ejerlejlighed/Villalejlighed       2
Rækkehus/Husbåd                    1
Landejendom/Fritidsbolig           1
Villalejlighed/Villa               1
Name: type, dtype: int64


In [57]:

df['type'].replace({
    'Ejerlejlighed/Fritidsbolig': 'Fritidsbolig',
    'Rækkehus/Fritidsbolig': 'Fritidsbolig',
    'Landejendom/Fritidsbolig': 'Fritidsbolig',
    'Rækkehus/Husbåd': 'Villa',
    'Landejendom/Villa': 'Villa',
    'Villa/Landejendom': 'Villa',
    'Villa/Helårsgrund ': 'Villa',
    'Villa/Helårsgrund': 'Villa',
    'Rækkehus/Villa': 'Villa',
    'Villalejlighed/Villa': 'Villa',
    'Ejerlejlighed/Villalejlighed': 'Ejerlejlighed',
    'Ejerlejlighed/Rækkehus': 'Ejerlejlighed'
}, inplace=True)

type_counts = df['type'].value_counts()
print(type_counts)

Villa                      2207
Ejerlejlighed               706
Rækkehus                    343
Landejendom                 173
Villa/Fritidsbolig          155
Villalejlighed               30
Fritidsbolig                 28
Landejendom/Landejendom       8
Name: type, dtype: int64


In [59]:
replacement_dict = {
    'Energimærke A2015': 'Energimærke A',
    'Energimærke A2020': 'Energimærke A',
    'Energimærke A1': 'Energimærke A',
    'Energimærke A2': 'Energimærke A'
}

df['energy'] = df['energy'].replace(replacement_dict)


In [66]:
df

,price,address,city,saledays,living_space,ground_space,rooms,owner_expenses,year,area_code,...,Villa,Villa/Fritidsbolig,Villalejlighed,Energimærke A,Energimærke B,Energimærke C,Energimærke D,Energimærke E,Energimærke F,Energimærke G
0,1275000,Strandparken 46,Vest- og Sydsjælland,202.0,170.0,1019.0,2.0,2.598,1978.0,4591,...,0,1,0,0,0,0,1,0,0,0
1,5095000,"Kalkbrænderihavnsgade 4A, 1. tv.",Byen København,0.0,94.0,0.0,3.0,2.389,2017.0,2100.0,...,0,0,0,1,0,0,0,0,0,0
2,14750000,Niels Andersens Vej 56,Københavns omegn,255.0,248.0,984.0,8.0,8.643,1941.0,2900.0,...,1,0,0,0,0,0,0,1,0,0
3,4498000,"Helga Pedersens Gade 1, 2. 3.",Østjylland,7.0,107.0,0.0,3.0,3.849,2014.0,8000.0,...,0,0,0,0,1,0,0,0,0,0
4,2850000,Nøddevænget 20,Sydjylland,0.0,163.0,858.0,5.0,3.343,1965.0,7100.0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,1750000,"Prins Haralds Allé 101, 1.",Fyn,48.0,109.0,506.0,3.0,1.922,1935.0,5250,...,0,0,1,0,0,0,0,0,0,1
3751,990000,Nissumvej 4,Vestjylland,48.0,117.0,800.0,3.0,1.566,1942.0,7620,...,0,1,0,0,0,1,0,0,0,0
3752,1695000,Baunetoften 12,Nordsjælland,240.0,86.0,103.0,4.0,2.184,1990.0,3320.0,...,0,0,0,0,0,1,0,0,0,0
3753,650000,Aalevej 40,Østjylland,48.0,79.0,829.0,3.0,1.464,1955.0,7160.0,...,1,0,0,0,0,0,1,0,0,0


In [61]:
type_dummies = pd.get_dummies(df['type'])
energy_dummies = pd.get_dummies(df['energy'])
df = pd.concat([df, type_dummies, energy_dummies], axis=1)

In [63]:
columns_to_drop = ['type', 'energy']
df = df.drop(columns=columns_to_drop)

In [64]:
df

,price,address,city,saledays,living_space,ground_space,rooms,owner_expenses,year,area_code,...,Villa,Villa/Fritidsbolig,Villalejlighed,Energimærke A,Energimærke B,Energimærke C,Energimærke D,Energimærke E,Energimærke F,Energimærke G
0,1275000,Strandparken 46,Vest- og Sydsjælland,202.0,170.0,1019.0,2.0,2.598,1978.0,"Havnsø,",...,0,1,0,0,0,0,1,0,0,0
1,5095000,"Kalkbrænderihavnsgade 4A, 1. tv.",Byen København,0.0,94.0,0.0,3.0,2.389,2017.0,2100,...,0,0,0,1,0,0,0,0,0,0
2,14750000,Niels Andersens Vej 56,Københavns omegn,255.0,248.0,984.0,8.0,8.643,1941.0,2900,...,1,0,0,0,0,0,0,1,0,0
3,4498000,"Helga Pedersens Gade 1, 2. 3.",Østjylland,7.0,107.0,0.0,3.0,3.849,2014.0,8000,...,0,0,0,0,1,0,0,0,0,0
4,2850000,Nøddevænget 20,Sydjylland,0.0,163.0,858.0,5.0,3.343,1965.0,7100,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,1750000,"Prins Haralds Allé 101, 1.",Fyn,48.0,109.0,506.0,3.0,1.922,1935.0,"Sanderum,",...,0,0,1,0,0,0,0,0,0,1
3751,990000,Nissumvej 4,Vestjylland,48.0,117.0,800.0,3.0,1.566,1942.0,"Nørlem,",...,0,1,0,0,0,1,0,0,0,0
3752,1695000,Baunetoften 12,Nordsjælland,240.0,86.0,103.0,4.0,2.184,1990.0,3320,...,0,0,0,0,0,1,0,0,0,0
3753,650000,Aalevej 40,Østjylland,48.0,79.0,829.0,3.0,1.464,1955.0,7160,...,1,0,0,0,0,0,1,0,0,0


In [65]:
# Step 1: Replace all non-numeric values in the "area_code" column with NaN
df['area_code'] = pd.to_numeric(df['area_code'], errors='coerce')

# Step 2: Extract numbers from the "area_name" column
df['extracted_area'] = df['area_name'].str.extract(r'(\d+)')

# Step 3: Fill "area_code" column based on conditions
df['area_code'] = df.apply(lambda row: row['extracted_area'] if pd.isnull(row['area_code']) else row['area_code'], axis=1)

# Drop the intermediate "extracted_area" column
df.drop('extracted_area', axis=1, inplace=True)

# Print the processed DataFrame
df

,price,address,city,saledays,living_space,ground_space,rooms,owner_expenses,year,area_code,...,Villa,Villa/Fritidsbolig,Villalejlighed,Energimærke A,Energimærke B,Energimærke C,Energimærke D,Energimærke E,Energimærke F,Energimærke G
0,1275000,Strandparken 46,Vest- og Sydsjælland,202.0,170.0,1019.0,2.0,2.598,1978.0,4591,...,0,1,0,0,0,0,1,0,0,0
1,5095000,"Kalkbrænderihavnsgade 4A, 1. tv.",Byen København,0.0,94.0,0.0,3.0,2.389,2017.0,2100.0,...,0,0,0,1,0,0,0,0,0,0
2,14750000,Niels Andersens Vej 56,Københavns omegn,255.0,248.0,984.0,8.0,8.643,1941.0,2900.0,...,1,0,0,0,0,0,0,1,0,0
3,4498000,"Helga Pedersens Gade 1, 2. 3.",Østjylland,7.0,107.0,0.0,3.0,3.849,2014.0,8000.0,...,0,0,0,0,1,0,0,0,0,0
4,2850000,Nøddevænget 20,Sydjylland,0.0,163.0,858.0,5.0,3.343,1965.0,7100.0,...,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3750,1750000,"Prins Haralds Allé 101, 1.",Fyn,48.0,109.0,506.0,3.0,1.922,1935.0,5250,...,0,0,1,0,0,0,0,0,0,1
3751,990000,Nissumvej 4,Vestjylland,48.0,117.0,800.0,3.0,1.566,1942.0,7620,...,0,1,0,0,0,1,0,0,0,0
3752,1695000,Baunetoften 12,Nordsjælland,240.0,86.0,103.0,4.0,2.184,1990.0,3320.0,...,0,0,0,0,0,1,0,0,0,0
3753,650000,Aalevej 40,Østjylland,48.0,79.0,829.0,3.0,1.464,1955.0,7160.0,...,1,0,0,0,0,0,1,0,0,0


In [67]:
csv_file_path = 'cleaned_data.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-16')

In [68]:
excel_file_path = 'cleaned_data_excel.xlsx'
df.to_excel(excel_file_path, index=False, encoding='utf-16')


c:\Users\Archie\anaconda3\lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
